# Imports y definiciones

In [2]:
import os, re, json
import numpy as np
import pandas as pd
import cvxpy as cp
from itertools import combinations
from ast import literal_eval
from collections import defaultdict

In [4]:
databases = 'C:/Users/Chris-Brota/Desktop/Databases'
display(os.listdir(databases))

# Se elige el JSON
db = f'{databases}/{os.listdir(databases)[1]}'
db

['foodb_2020_04_07_csv',
 'foodb_2020_04_07_json',
 'foodb_server_dump_2020_4_21.sql',
 'FoodData_Central_foundation_food_csv_2021-04-28']

'C:/Users/Chris-Brota/Desktop/Databases/foodb_2020_04_07_json'

# Tablas a sacar

Estas tablas en general no poseen mucha de la información relevante que se necesita, por lo cual no se considerará en el MVP0. 

In [5]:
dont_consider = (109, 168, 219, 905, 947)

# Características de interés

In [6]:
compounds_mvp0 = ['Protein', 'Calcium, Ca', 'Iron, Fe', 'Phosphorus, P', 
                  'Potassium, K', 'Sodium, Na', 'Zinc, Zn', 'Copper, Cu', 
                  'Thiamin', 'Riboflavin', 'Niacin', 'Vitamin B6', 'Folic acid']

# Creación de las recetas

In [7]:
# Número de ingredientes máximos por receta
compounds_max = 5

# Definición de los ingredientes
ingredients = os.listdir('summary/foodTables')

# Definición de las diferentes combinaciones de recetas en base a los ingredientes
recipes = combinations(ingredients, compounds_max)

In [8]:
for recipe_i in recipes:
    recipe_to = recipe_i
    print(recipe_i)
#     break

('159_Rosemary.txt', '165_Common sage.txt', '183_Common thyme.txt', '206_Ginger.txt', '246_Triticale.txt')
('159_Rosemary.txt', '165_Common sage.txt', '183_Common thyme.txt', '206_Ginger.txt', '341_Coconut.txt')
('159_Rosemary.txt', '165_Common sage.txt', '183_Common thyme.txt', '206_Ginger.txt', '372_Ginkgo nuts.txt')
('159_Rosemary.txt', '165_Common sage.txt', '183_Common thyme.txt', '206_Ginger.txt', '384_Horseradish tree.txt')
('159_Rosemary.txt', '165_Common sage.txt', '183_Common thyme.txt', '206_Ginger.txt', '40_Pepper.txt')
('159_Rosemary.txt', '165_Common sage.txt', '183_Common thyme.txt', '206_Ginger.txt', '562_Shiitake.txt')
('159_Rosemary.txt', '165_Common sage.txt', '183_Common thyme.txt', '206_Ginger.txt', '569_Maitake.txt')
('159_Rosemary.txt', '165_Common sage.txt', '183_Common thyme.txt', '206_Ginger.txt', '586_Cinnamon.txt')
('159_Rosemary.txt', '165_Common sage.txt', '183_Common thyme.txt', '206_Ginger.txt', '58_Coffee.txt')
('159_Rosemary.txt', '165_Common sage.txt'

('183_Common thyme.txt', '341_Coconut.txt', '40_Pepper.txt', '68_Turmeric.txt', '748_Hibiscus tea.txt')
('183_Common thyme.txt', '341_Coconut.txt', '562_Shiitake.txt', '569_Maitake.txt', '586_Cinnamon.txt')
('183_Common thyme.txt', '341_Coconut.txt', '562_Shiitake.txt', '569_Maitake.txt', '58_Coffee.txt')
('183_Common thyme.txt', '341_Coconut.txt', '562_Shiitake.txt', '569_Maitake.txt', '666_Salt.txt')
('183_Common thyme.txt', '341_Coconut.txt', '562_Shiitake.txt', '569_Maitake.txt', '68_Turmeric.txt')
('183_Common thyme.txt', '341_Coconut.txt', '562_Shiitake.txt', '569_Maitake.txt', '748_Hibiscus tea.txt')
('183_Common thyme.txt', '341_Coconut.txt', '562_Shiitake.txt', '586_Cinnamon.txt', '58_Coffee.txt')
('183_Common thyme.txt', '341_Coconut.txt', '562_Shiitake.txt', '586_Cinnamon.txt', '666_Salt.txt')
('183_Common thyme.txt', '341_Coconut.txt', '562_Shiitake.txt', '586_Cinnamon.txt', '68_Turmeric.txt')
('183_Common thyme.txt', '341_Coconut.txt', '562_Shiitake.txt', '586_Cinnamon.txt

In [24]:
recipe_to

('159_Rosemary.txt',
 '165_Common sage.txt',
 '183_Common thyme.txt',
 '206_Ginger.txt',
 '341_Coconut.txt')

In [26]:
# Definición de la tabla nutricional objetivo
obj_table = list()

with open('summary/objective_table.csv', 'r', encoding='utf8') as file:
    # Saltar el encabezado
    file.readline()
    
    for line in file:
        compound_to_rev = line.strip().split(';')[0].replace('\"', '')
        
        if compound_to_rev in compounds_mvp0:
            obj_table.append((compound_to_rev, float(line.strip().split(';')[1])))

obj_table

[('Protein', 0.002557603686635945),
 ('Calcium, Ca', 5.357142857142858e-05),
 ('Iron, Fe', 2.1601382488479263e-07),
 ('Phosphorus, P', 1.2857142857142856e-05),
 ('Potassium, K', 3.785714285714286e-05),
 ('Sodium, Na', 0.0004450000000000001),
 ('Zinc, Zn', 9.504608294930875e-07),
 ('Copper, Cu', 2.9665898617511523e-08),
 ('Thiamin', 1.759792626728111e-07),
 ('Riboflavin', 3.185195852534562e-06),
 ('Niacin', 4.606019585253456e-05),
 ('Vitamin B6', 9.978398617511522e-06),
 ('Folic acid', 2.385714285714286e-07)]

In [29]:
# Creación del problema de optimización para la definición de la receta
recipe_matrix = list()

for ing_i in recipe_to:
    # Definición de la información del ingrediente i
    ing_i_list = list()
    
    with open(f'summary/foodTables/{ing_i}', 'r', encoding='utf8') as file:
        for line in file:
            comp_i, value_i = line.strip().split(';')
            ing_i_list.append((comp_i, float(value_i)))
            
    # Agregando a la matriz de recetas
    recipe_matrix.append(ing_i_list)

In [30]:
recipe_matrix

[[('Protein', 0.044511333333333326),
  ('Calcium, Ca', 0.008168625),
  ('Iron, Fe', 0.000183875),
  ('Phosphorus, P', 0.00069125),
  ('Potassium, K', 0.00824325),
  ('Sodium, Na', 0.00038675),
  ('Zinc, Zn', 2.1224999999999998e-05),
  ('Copper, Cu', 4.255000000000001e-06),
  ('Thiamin', 1.9533333333333334e-06),
  ('Riboflavin', 2.8999999999999998e-06),
  ('Niacin', 9.685e-06),
  ('Vitamin B6', 8.040000000000001e-06),
  ('Folic acid', 0.00027354)],
 [('Protein', 0.13376666666666667),
  ('Calcium, Ca', 0.01544925),
  ('Iron, Fe', 0.00025435),
  ('Phosphorus, P', 0.0010025),
  ('Potassium, K', 0.015521666666666668),
  ('Sodium, Na', 0.00040050000000000003),
  ('Zinc, Zn', 5.3e-05),
  ('Copper, Cu', 7.535e-06),
  ('Thiamin', 7.54e-06),
  ('Riboflavin', 3.43e-06),
  ('Niacin', 5.8349999999999995e-05),
  ('Vitamin B6', 2.69e-05),
  ('Folic acid', 2.0550000000000002e-06)],
 [('Protein', 0.0819),
  ('Calcium, Ca', 0.013722416666666667),
  ('Iron, Fe', 0.0008225),
  ('Phosphorus, P', 0.00153208

# Transformando a arrays

In [40]:
obj_table_arr = np.array(obj_table)[:,1].astype(float)
recipe_matrix_arr = np.array(recipe_matrix)[:,:,1].astype(float).T

In [41]:
print(recipe_matrix_arr.shape)
print(obj_table_arr.shape)

(13, 5)
(13,)


# Planteando el modelo de optimización

In [88]:
# Variable de cantidad
x = cp.Variable(compounds_max)

# Restricciones
constraints = [x >= 0]

# Definición de la función objetivo
objective_func = cp.Minimize(cp.sum_squares((obj_table_arr - recipe_matrix_arr @ x)))

# Definición del problema de optimización
prob = cp.Problem(objective=objective_func, constraints=constraints)

In [102]:
# Calculando el óptimo
result = prob.solve(solver=cp.ECOS, verbose=True)


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -9.335e-18  -4.339e-01  +8e+00  7e-01  5e-01  1e+00  1e+00    ---    ---    1  1  - |  -  - 
 1  -3.387e-02  -7.591e-02  +7e-01  3e-02  4e-02  6e-02  1e-01  0.9159  1e-02   1  1  1 |  0  0
 2  +3.588e-03  -6.677e-03  +2e-01  7e-03  7e-03  8e-03  3e-02  0.7728  1e-02   1  1  1 |  0  0
 3  -5.124e-03  -9.866e-03  +1e-01  4e-03  2e-03  3e-03  2e-02  0.6892  4e-01   1  1  1 |  0  0
 4  +4.085e-04  -5.075e-04  +2e-02  7e-04  4e-04  6e-04  3e-03  0.9890  2e-01   1  1  1 |  0  0
 5  -6.703e-05  -2.078e-04  +3e-03  1e-04  5e-05  5e-05  5e-04  0.9112  7e-02   1  1  1 |  0  0
 6  +4.645e-06  -3.753e-05  +9e-04  3e-05  1e-05  2e-05  2e-04  0.8149  2e-01   1  1  1 |  0  0
 7  -1.749e-05  -3.508e-05  +4e-04  1e-05  6e-06  7e-06  7e-05  0.7762  3e-01   1  1  1 |  0  0
 8  -2.237e-07  -3.778e-06  +8e-05  3e-06  1e-

In [98]:
print(result)
print(x.value)

2.4020366419017626e-07
[2.75837536e-05 4.47637131e-05 4.87446358e-03 3.79129352e-05
 9.01233271e-02]


In [99]:
obj_table_arr

array([2.55760369e-03, 5.35714286e-05, 2.16013825e-07, 1.28571429e-05,
       3.78571429e-05, 4.45000000e-04, 9.50460829e-07, 2.96658986e-08,
       1.75979263e-07, 3.18519585e-06, 4.60601959e-05, 9.97839862e-06,
       2.38571429e-07])

In [100]:
recipe_matrix_arr @ x.value

array([2.53051968e-03, 7.78267203e-05, 5.45722224e-06, 8.77620543e-05,
       2.94117453e-04, 3.81046608e-05, 8.48245543e-07, 3.41180899e-07,
       3.87754949e-08, 8.22074092e-08, 6.59883193e-07, 9.14577988e-08,
       3.81171197e-06])